# Shor's Algorithm (Manual Implementation)

This notebook demonstrates a manual implementation of Shor's Algorithm using Qiskit without relying on the built-in `Shor` class.

In [78]:
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT
from math import gcd, ceil, log
from fractions import Fraction
import numpy as np


In [2]:
def qpe_modexp(a: int, N: int, n_count: int) -> QuantumCircuit:
    qc = QuantumCircuit(n_count + 4, n_count)
    for q in range(n_count):
        qc.h(q)
    qc.x(n_count)
    for q in range(n_count):
        qc = apply_c_amod15(qc, a**(2**q) % N, q, n_count)
    qc.append(QFT(num_qubits=n_count, inverse=True, do_swaps=True), range(n_count))
    qc.measure(range(n_count), range(n_count))
    return qc


In [3]:
def apply_c_amod15(qc, a: int, control_qubit: int, n_count: int) -> QuantumCircuit:
    x = n_count
    if a == 2:
        qc.cswap(control_qubit, x, x+1)
        qc.cswap(control_qubit, x+1, x+2)
        qc.cswap(control_qubit, x+2, x+3)
    elif a == 4:
        qc.cswap(control_qubit, x, x+2)
        qc.cswap(control_qubit, x+1, x+3)
    elif a == 7:
        qc.cswap(control_qubit, x, x+3)
        qc.cswap(control_qubit, x+1, x+2)
    elif a == 8:
        qc.cswap(control_qubit, x, x+2)
        qc.cswap(control_qubit, x+1, x+3)
        qc.cswap(control_qubit, x+2, x+3)
    elif a == 11:
        qc.cswap(control_qubit, x, x+1)
        qc.cswap(control_qubit, x+1, x+2)
        qc.cswap(control_qubit, x+2, x+3)
    elif a == 13:
        qc.cswap(control_qubit, x, x+2)
        qc.cswap(control_qubit, x+1, x+3)
        qc.cswap(control_qubit, x+2, x+3)
    return qc


In [4]:
def get_period(phase: int, n_count: int) -> int:
    decimal = phase / (2 ** n_count)
    frac = Fraction(decimal).limit_denominator(15)
    return frac.denominator


In [71]:
def get_circuit0(a,N):
    """ Get an integer a that is coprime with N """
    #a = get_value_a(N)

    """ If user wants to force some values, can do that here, please make sure to update print and that N and a are coprime"""
    """print('Forcing N=15 and a=4 because its the fastest case, please read top of source file for more info')
    N=15
    a=2"""

    """ Get n value used in Shor's algorithm, to know how many qubits are used """
    n = ceil(log(N,2))
    
    """ Create quantum and classical registers """

    """auxilliary quantum register used in addition and multiplication"""
    aux = QuantumRegister(n+2)
    """single qubit where the sequential QFT is performed"""
    up_reg = QuantumRegister(1)
    """quantum register where the multiplications are made"""
    down_reg = QuantumRegister(n)
    """classical register where the measured values of the sequential QFT are stored"""
    up_classic = ClassicalRegister(2*n)
    """classical bit used to reset the state of the top qubit to 0 if the previous measurement was 1"""
    c_aux = ClassicalRegister(1)

    """ Create Quantum Circuit """
    circuit = QuantumCircuit(down_reg , up_reg , aux, up_classic, c_aux)

    """ Initialize down register to 1"""
    circuit.x(down_reg[0])

    """ Cycle to create the Sequential QFT, measuring qubits and applying the right gates according to measurements """
    for i in range(0, 2*n):
        """reset the top qubit to 0 if the previous measurement was 1"""
        circuit.x(up_reg).c_if(c_aux, 1)
        circuit.h(up_reg)
        cMULTmodN(circuit, up_reg[0], down_reg, aux, a**(2**(2*n-1-i)), N, n)
        """cycle through all possible values of the classical register and apply the corresponding conditional phase shift"""
        for j in range(0, 2**i):
            """the phase shift is applied if the value of the classical register matches j exactly"""
            circuit.u1(getAngle(j, i), up_reg[0]).c_if(up_classic, j)
        circuit.h(up_reg)
        circuit.measure(up_reg[0], up_classic[i])
        circuit.measure(up_reg[0], c_aux[0])
    return circuit
    

In [75]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from math import ceil, log, gcd
import numpy as np

def get_circuit(a, N):
    """ Get an integer a that is coprime with N """
    n = ceil(log(N, 2))

    """ Create quantum and classical registers """
    aux = QuantumRegister(n+2, name="aux")
    up_reg = QuantumRegister(1, name="up")
    down_reg = QuantumRegister(n, name="down")
    up_classic = ClassicalRegister(2*n, name="up_classic")
    c_aux = ClassicalRegister(1, name="c_aux")

    """ Create Quantum Circuit """
    circuit = QuantumCircuit(down_reg, up_reg, aux, up_classic, c_aux)

    """ Initialize down register to 1"""
    circuit.x(down_reg[0])

    """ Cycle to create the Sequential QFT, measuring qubits and applying the right gates according to measurements """
    #for i in range(0, 2*n):
    #    """ Reset the top qubit to 0 if the previous measurement was 1 """
    #    circuit.x(up_reg).c_if(c_aux, 1)
    #    circuit.h(up_reg)
    #    
    #    # Replace deprecated function calls
    #    cMULTmodN(circuit, up_reg[0], down_reg, aux, a**(2**(2*n-1-i)), N, n)
    #    
    #    """ Apply conditional phase shift """
    #    for j in range(0, 2**i):
    #        circuit.p(getAngle(j, i), up_reg[0]).c_if(up_classic, j)  # Replace u1 with p
    #
    #    circuit.h(up_reg)
    #    circuit.measure(up_reg[0], up_classic[i])
    #    circuit.measure(up_reg[0], c_aux[0])

    for i in range(0, 2*n):
        """ Reset the top qubit to 0 if the previous measurement was 1 """
        reset_instruction = circuit.x(up_reg[0])  # Store instruction
        reset_instruction.c_if(c_aux, 1)  # Apply conditional logic

        circuit.h(up_reg[0])

        # Replace deprecated function calls
        cMULTmodN(circuit, up_reg[0], down_reg, aux, a**(2**(2*n-1-i)), N, n)

        """ Apply conditional phase shift """
        for j in range(0, 2**i):
            phase_instruction = circuit.p(getAngle(j, i), up_reg[0])  # Store instruction
            phase_instruction.c_if(up_classic, j)  # Apply conditional logic

        circuit.h(up_reg[0])
        circuit.measure(up_reg[0], up_classic[i])
        circuit.measure(up_reg[0], c_aux[0])

    return circuit


In [64]:
def get_random_base(N):
    base = np.random.randint(2, N)
    while gcd(base, N) != 1:
        base = np.random.randint(2, N)
    return base

In [65]:
def shor_manual(N: int, a: int = 7) -> int:
    if gcd(a, N) != 1:
        return gcd(a, N)

    n_count = 8
    #qc = qpe_modexp(a, N, n_count)
    qc =  get_circuit(a,N)
    #backend = Aer.get_backend('qasm_simulator')
    #result = execute(qc, backend, shots=1).result()

    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    job = simulator.run(compiled_circuit)
    result = job.result()
    
    counts = result.get_counts()
    print(counts)
    phase_bin = max(counts, key=counts.get)
    phase_int = int(phase_bin, 2)
    r = get_period(phase_int, n_count)
    if r % 2 != 0:
        return None
    plus = pow(a, r // 2) + 1
    minus = pow(a, r // 2) - 1
    factor1 = gcd(plus, N)
    factor2 = gcd(minus, N)
    if factor1 == 1 or factor1 == N:
        return None
    return factor1


In [76]:
N = 35
a = get_random_base(N)
factor = None
while factor == None :
    a = get_random_base(N)
    factor = shor_manual(N, a)
print(f"Found factor of {N} using base {a}: {factor}")


AttributeError: 'InstructionSet' object has no attribute 'c_if'

In [52]:
11*13


143

In [54]:
N = 143
a = get_random_base(N)
for q in range(8):
    print(a**(2**q) % N)

138
25
53
92
27
14
53
92


In [80]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

import numpy as np
from math import gcd

def get_random_base(N):
    """ Select a random base that is coprime with N """
    base = np.random.randint(2, N)
    while gcd(base, N) != 1:
        base = np.random.randint(2, N)
    return base

def shors_algorithm(N):
    base = get_random_base(N)
    print(f"Random base selected: {base}")

    # Quantum circuit for period finding
    circuit = QuantumCircuit(4, 4)
    circuit.h(range(4))  # Apply Hadamard gates
    # Add modular exponentiation and measurement...

    #simulator = Aer.get_backend('qasm_simulator')
    #result = execute(circuit, simulator).result()

    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    job = simulator.run(compiled_circuit)
    result = job.result()

    # Extract period and compute factors (simplified here)
    factors = [gcd(base**2 - 1, N), gcd(base**2 + 1, N)]
    return factors

N = 35  # Number to factor
print(shors_algorithm(N))

Random base selected: 3


NameError: name 'qc' is not defined

In [82]:
n =15

n.bit_length()

4